In [1]:
import os
import pandas as pd

import dask.dataframe as dd

In [2]:
DATA_DIR = '../../data/raw'
LOOKUPS = DATA_DIR + "/Lookups"
TRANSACT = DATA_DIR + "/Transactions"

In [3]:
files = os.listdir(TRANSACT)

In [9]:
stores = pd.read_csv(LOOKUPS + '/stores.txt', sep='\t')
drugs = pd.read_csv(LOOKUPS + '/Drug_LookUp.txt', sep='\t')
illness = pd.read_csv(LOOKUPS + '/ChronicIllness_LookUp.txt', sep='\t')
patients = pd.read_csv(LOOKUPS + '/patients.txt', sep='\t')
atc = pd.read_csv(LOOKUPS + '/ATC_LookUp.txt', sep='\t')

In [10]:
t1 = illness.merge(drugs, on='MasterProductID', how='outer')

In [5]:
def get_subset():

    samples = []
    for file in files:
        df = pd.read_csv(TRANSACT + '/' + file, sep='\t')
        sample = df.sample(frac=.1)
        samples.append(sample)

    return pd.concat(samples)
    
    

In [6]:
subset = get_subset()

In [11]:
subset = subset.merge(t1, left_on='Drug_ID', right_on='MasterProductID', suffixes=('_illness', '_drug'), how='outer')

In [13]:
subset = subset.merge(patients, on='Patient_ID', how='outer')

In [14]:
subset['target'] = subset['ChronicIllness'] == 'Diabetes'

In [15]:
subset.head()

,Patient_ID,Store_ID,Prescriber_ID,Drug_ID,SourceSystem_Code,Prescription_Week,Dispense_Week,Drug_Code,NHS_Code,IsDeferredScript,...,ChemistListPrice,ATCLevel5Code,ATCLevel4Code,ATCLevel3Code,ATCLevel2Code,ATCLevel1Code,gender,year_of_birth,postcode,target
0,7716.0,2636.0,6951.0,6470.0,F,2012-09-09,2013-02-17,PANA8,8814X,0.0,...,4.44,N02BE01,N02BE,N02B,N02,N,M,1900.0,2648.0,False
1,7716.0,2636.0,6951.0,6470.0,F,2013-06-23,2013-07-21,PANA8,8814X,0.0,...,4.44,N02BE01,N02BE,N02B,N02,N,M,1900.0,2648.0,False
2,7716.0,2636.0,6951.0,6470.0,F,2011-06-19,2011-07-24,PANA8,8814X,0.0,...,4.44,N02BE01,N02BE,N02B,N02,N,M,1900.0,2648.0,False
3,7716.0,2636.0,6951.0,2444.0,F,2013-08-25,2013-10-27,DIAM2,9302N,0.0,...,9.19,A10BB09,A10BB,A10B,A10,A,M,1900.0,2648.0,True
4,7716.0,2636.0,6951.0,9008.0,F,2013-03-03,2013-05-26,VENT3,8288F,0.0,...,4.97,R03AC02,R03AC,R03A,R03,R,M,1900.0,2648.0,False
